In [2]:
# import torch

# # CUDA major, minor 버전 확인
# major_version, minor_version = torch.cuda.get_device_capability()
# major_version, minor_version

# # unsloth 설치
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# if major_version >= 8:
#     # 새로운 GPU(예: Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)에 사용
#     !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
# else:
#     # 오래된 GPU(예: V100, Tesla T4, RTX 20xx)에 사용하세요.
#     !pip install --no-deps xformers trl peft accelerate bitsandbytes
# pass

In [1]:
# !pip install torch
# !pip install langchain
# !pip install langchain-community
# !pip install pymupdf
# !pip install chromadb
# !pip install transformers
# !pip install sentence-transformers

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# 단계 1: 문서 로드(Load Documents)
PDF_FILE_PATH = "SPRi AI Brief 5월호 산업동향.pdf"
try:
    loader = PyMuPDFLoader(PDF_FILE_PATH)
    docs = loader.load()
except Exception as e:
    print(f"Error loading PDF: {e}")
    print(f"Please ensure the PDF file is available at '{PDF_FILE_PATH}' or provide a direct downloadable URL.")
    docs = [] 

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = HuggingFaceEmbeddings(
    model_name="BM-K/KoSimCSE-roberta-multitask"
)

# 단계 4: DB 생성(Create DB) 및 저장
vectorstore = Chroma.from_documents(
    documents=split_documents,
    embedding=embeddings,
    persist_directory="./chroma_db",
    collection_metadata={"hnsw:construction_ef": 500, "hnsw:M": 32},
)

# 단계 5: 검색기(Retriever) 생성
# 변경: CrossEncoderReranker에서 top_k 제거, ContextualCompressionRetriever에서 k=3 설정
base_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        'k': 10,
        'fetch_k': 20,
        'score_threshold': 0.6
    }
)
cross_encoder = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
reranker = CrossEncoderReranker(model=cross_encoder)  # 변경: top_k=3 제거
retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=base_retriever,
    search_kwargs={"k": 3}  # 추가: 상위 3개 문서 반환 설정
)

# 단계 6: 프롬프트 생성(Create Prompt)
system_prompt_text = """당신은 주어진 컨텍스트를 기반으로 질문에 답변하는 AI 어시스턴트입니다.
답변은 간결하고 정확해야 하며, 한국어로 작성하세요.
컨텍스트에 없는 정보는 절대로 사용하지 말고, 만약 컨텍스트에 답변이 없다면 "컨텍스트에서 답변을 찾을 수 없습니다."라고 솔직하게 답변하세요."""

llama3_prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id>

#Context:
{context}

#Question:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id>
"""

prompt = PromptTemplate(
    template=llama3_prompt_template,
    input_variables=["context", "question"],
    partial_variables={"system_prompt": system_prompt_text}
)

# 단계 7: 언어모델(LLM) 생성
model_name = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
    )
except Exception as e:
    print(f"Error loading model {model_name}: {e}")
    raise e

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token 

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    return_full_text=False,
     max_new_tokens=256,  # 변경: 256 -> 1024로 증가하여 더 긴 답변 생성
    repetition_penalty=1.2,  # 추가: 긴 답변에서 반복 억제 강화
    top_p=0.95  # 변경: 0.9 -> 0.95로 조정하여 긴 답변에서도 다양성 유지
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever | RunnableLambda(format_docs), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.41it/s]
Device set to use cuda:0


In [7]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "EU집행위원회는 AI에 얼마를 투자했지?"
response = chain.invoke(question)
print(response)

/home/elicer/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


EU 집행위원회는 'InvestAI' 이니셔티브를 통해 AI에 총 2,000억 유로를 투자했습니다. 또한, AI 대륙 행동계획에서는 AI 컴퓨팅 인프라 구축, 데이터 접근성 확대, 전략적 영역의 AI 촉진, AI 역량과 인재 육성, AI 법 시행 간소화 등의 5가지 주요 과제를 제시하여 AI 리더십을 강조합니다. AI 대륙 행동계획에는 AI 컴퓨팅 인프라 구축을 위하여 200억 유로를 투자하겠다고 밝혔다. 더불어, EU 내 데이터센터 용량을 최소 3배 확대하고, 데이터 센터 관련 허가 절차 간소화 등을 통해서 클라우드와 데이터센터 투자 활성을 장려할 방침입니다.󰏽️🤝‍♀️😊️💬️👍️🎯️🚫️🕒️📈️🛠️👸️👋️👂️👆️👇️👅️�


In [5]:
# !rm -rf chroma_db